# First imports

In [ ]:
# Import necessary libraries for data manipulation, numerical operations, and visualization
import pandas as pd
import seaborn as sns
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

In [ ]:
# specify the path of the files 
file_path = 'content/*.csv'

# Use glob to get the list of all corresponding files
all_files = glob.glob(file_path)

# Inicialize an empty list to storage the dataframes
dfs = []

# Iterate over the list of files and read each one into a DataFrame
for file in all_files:
    df = pd.read_csv(file, sep=';', encoding='latin1')  
    dfs.append(df)

# Combine all the dataframes in one
fuels_data = pd.concat(dfs, ignore_index=True)

# show the first lines of the combined dataframe
fuels_data.head()


In [ ]:
# Display the rows and columns of the dataframe
fuels_data.shape

# Data Manipulation

In [ ]:
# display the type of the columns
fuels_data.info()

In [ ]:
# Apllying lambda to replace all the ',' to '.' at the row 'Valor de Venda'
fuels_data['Valor de Venda'] = fuels_data['Valor de Venda'].apply(lambda x: str(x).replace(',', '.'))

In [ ]:
# Change the datatype of 'valor de venda' to float64
fuels_data['Valor de Venda'] = fuels_data['Valor de Venda'].astype(np.float64)

In [ ]:
# List of columns to delete
columns_to_delete = ['ï»¿Regiao - Sigla', 'Municipio', 'Revenda', 'CNPJ da Revenda', 'Nome da Rua', 'Numero Rua', 'Complemento', 'Bairro', 'Cep', 'Valor de Compra', 'Bandeira', 'Regiao - Sigla']

In [ ]:
# Deleting those columns above
fuels_data = fuels_data.drop(columns= columns_to_delete, axis=1).copy()

In [ ]:
fuels_data.head()

In [ ]:
# Display all the unique data of the row 'Unidade de Medida'
fuels_data['Unidade de Medida'].unique()

In [ ]:
# Only use the rows where 'Unidade de Medida' equals 'R$ / litro'
fuels_data = fuels_data[fuels_data['Unidade de Medida'] == 'R$ / litro']

In [ ]:
# Change the datatype of 'Data da coleta' to datetime 'dd/mm/yyyy
fuels_data['Data da Coleta'] = pd.to_datetime(fuels_data['Data da Coleta'], format='%d/%m/%Y')

In [ ]:
# Change the format to 'mm-yyyy'
fuels_data['Data da Coleta'] = fuels_data['Data da Coleta'].dt.strftime('%m-%Y')

In [ ]:
# Display the rows and columns of the dataframe
fuels_data.shape

In [ ]:
# Display the unique values at the row 'Produto'
fuels_data['Produto'].unique()

Gas data select

In [ ]:
# Selects only the rows where 'produto' equals 'gasolina'
gas_data = fuels_data[fuels_data['Produto'] == 'GASOLINA']
gas_data.head()

In [ ]:
gas_data.shape

ethanol data select

In [ ]:
# # Selects only the rows where 'produto' equals 'etanol'
ethanol_data = fuels_data[fuels_data['Produto'] == 'ETANOL']
ethanol_data.head()

In [ ]:
ethanol_data.shape

# Minimum wage

In [ ]:
# Reads a html file
minimum_wage = pd.read_html("Brazil_minimum_wage.html")

In [ ]:
# Display the lenght of the list of dataframes at the html
len(minimum_wage)

In [ ]:
# Display the first dataframe
minimum_wage[0]    

In [ ]:
# Display the second dataframe
minimum_wage[1]

In [ ]:
# Display the third dataframe
minimum_wage[2]

In [ ]:
# Choosing the third dataframe
minimum_wage = minimum_wage[2]

In [ ]:
# Dropping the first row of the dataframe (old header)
minimum_wage = minimum_wage.drop(0, axis=0)

In [ ]:
# dropping some columns
minimum_wage = minimum_wage.drop([0, 3, 4], axis=1)

In [ ]:
# Dropping the last rows (unnecessary)
minimum_wage = minimum_wage.drop([32, 33, 34, 35, 36], axis=0)

In [ ]:
minimum_wage

In [ ]:
# Defining the new names of the columns
minimum_wage.columns = ['Data', 'Valor']

In [ ]:
# Apply a Lambda method to replace the ',' to '.' at the columns 'Valor'
minimum_wage['Valor'] = minimum_wage['Valor'].apply(lambda x: x.replace(',', '.'))

In [ ]:
# Reseting the index at the wage datafrme
minimum_wage = minimum_wage.reset_index(drop=True)

In [ ]:
# Manually fixing the 6th row to it's correct value 
minimum_wage['Valor'][5] = 'R$ 1.045.00'

In [ ]:
minimum_wage

**Applying a replace using regex at the column 'Valor' to remove unescessary text (we only want the integer value of wage)**

In [ ]:
minimum_wage['Valor'] = minimum_wage['Valor'].str.replace(r'R\$ \d+\.\d{2} com reajuste em \w+ para R\$', '', regex=True)

In [ ]:
minimum_wage['Valor'] = minimum_wage['Valor'].str.replace('R$', '')

In [ ]:
minimum_wage['Valor'] = minimum_wage['Valor'].str.replace('.00', '')

In [ ]:
minimum_wage['Valor'] = minimum_wage['Valor'].str.replace('.', '')

In [ ]:
minimum_wage['Valor'] = minimum_wage['Valor'].astype(np.float64)

In [ ]:
minimum_wage.head(40)

## Grouping per year

**Formating the 'Data' column correctly so we can group the values per year only**

In [ ]:
# Changing the format of the column 'Data' 
minimum_wage['Data'] = minimum_wage['Data'].str.replace('/', '-')

In [ ]:
minimum_wage['Data'] = pd.to_datetime(minimum_wage['Data'], format='%m-%Y')

In [ ]:
minimum_wage['Data'] = minimum_wage['Data'].dt.year

In [ ]:
minimum_wage = minimum_wage[minimum_wage['Data'] > 2003]

In [ ]:
minimum_wage = minimum_wage[minimum_wage['Data'] != 2024]

In [ ]:
minimum_wage = minimum_wage[minimum_wage['Valor'] != 1302]

In [ ]:
minimum_wage = minimum_wage.reset_index(drop=True)

In [ ]:
minimum_wage

# grouping gas data

In [ ]:
gas_data.head()

In [ ]:
# Saving the gas_data dataframe to gas_price where we will group 'Valor de Venda' per 'Data da Coleta'
gas_price = gas_data.copy()
gas_price.head()

In [ ]:
# Grouping the 'Valor de Venda' column per 'Data da Coleta'
gas_price = gas_data.groupby('Data da Coleta')['Valor de Venda'].mean()

In [ ]:
# Fixing the index
gas_price = gas_price.reset_index()

In [ ]:
# Changing the format to datetime mm-yyyy
gas_price['Data da Coleta'] = pd.to_datetime(gas_price['Data da Coleta'], format='%m-%Y')

In [ ]:
# Setting the new column names
gas_price.columns = ['Data', 'Valor']

In [ ]:
# Grouping again per Data, but now it's datetime type
gas_price = gas_price.groupby('Data')['Valor'].mean()

In [ ]:
# Sorting by index
gas_price = gas_price.sort_index(ascending=False)

In [ ]:
# Fixing the index
gas_price = gas_price.reset_index()

In [ ]:
gas_price.head()

# Grouping ethanol data

In [ ]:
ethanol_data.head()

In [ ]:
# Saving the ethanol_data dataframe to ethanol_price where we will group 'Valor de Venda' per 'Data da Coleta'

ethanol_price = ethanol_data.copy()
ethanol_price.head()

In [ ]:
# Grouping the 'Valor de Venda' column per 'Data da Coleta'
ethanol_price = ethanol_data.groupby('Data da Coleta')['Valor de Venda'].mean()

In [ ]:
# Fixing the index of the dataframe
ethanol_price = ethanol_price.reset_index()

In [ ]:
# Changing the format to datetime mm-yyyy
ethanol_price['Data da Coleta'] = pd.to_datetime(ethanol_price['Data da Coleta'], format='%m-%Y')

In [ ]:
# Setting the new column names
ethanol_price.columns = ['Data', 'Valor']

In [ ]:
# Grouping again per Data, but now it's datetime type
ethanol_price = ethanol_price.groupby('Data')['Valor'].mean()

In [ ]:
# Sorting by index
ethanol_price = ethanol_price.sort_index(ascending=False)

In [ ]:
# Fixing the index
ethanol_price = ethanol_price.reset_index()

In [ ]:
ethanol_price.head()

# Liters of gas per wage

In [ ]:
gas_price.head()

In [ ]:
minimum_wage

In [ ]:
# Creating a new dataframe to show the liters of gas per wage correlation
gas_correlation = gas_price.copy()

In [ ]:
# Creating a new column to storage the Year of the column 'Data'
gas_correlation['Ano'] = gas_correlation['Data'].dt.year

In [ ]:
# Merging the 'Gas_correlation' to the 'Minimum_wage' Dataframe Using the 'Ano' and 'Data' values
df_merged = gas_correlation.merge(minimum_wage, left_on='Ano', right_on='Data', how='left', suffixes=('', '_Anual'))

In [ ]:
df_merged

In [ ]:
gas_correlation

In [ ]:
# Inserting the 'Valor_Anual' column to 'Gas_correlation' Dataframe
gas_correlation['Wage'] = df_merged['Valor_Anual']

In [ ]:
# Defining the new column names of the dataframe
gas_correlation.columns = ['Data', 'Gas', 'Ano', 'Wage']

In [ ]:
gas_correlation 

In [ ]:
# removing the 'Ano' column (unescessary)
gas_correlation.drop('Ano', axis=1, inplace=True)

In [ ]:
# Getting the number of Liters of gas one can afford with one minimum wage
gas_correlation['liters per wage'] = gas_correlation['Wage'] / gas_correlation['Gas']

In [ ]:
gas_correlation

In [ ]:
# Create a new dataframe with the 'Data' and 'liters per wage' column
gas_per_wage = gas_correlation[['Data', 'liters per wage']]

In [ ]:
gas_per_wage

# Liters of ethanol per wage

In [ ]:
ethanol_price.head()

In [ ]:
minimum_wage.head()

In [ ]:
# Creating a new dataframe to show the liters of ethanol per wage correlation
ethanol_correlation = ethanol_price.copy()

In [ ]:
ethanol_correlation

In [ ]:
# Creating a new column to storage the Year of the column 'Data'
ethanol_correlation['Ano'] = gas_correlation['Data'].dt.year

In [ ]:
# Merging the 'ethanol_correlation' to the 'Minimum_wage' Dataframe Using the 'Ano' and 'Data' values
df_merged = ethanol_correlation.merge(minimum_wage, left_on='Ano', right_on='Data', how='left', suffixes=('', '_Anual'))

In [ ]:
# Inserting the 'Valor_Anual' column to 'ethanol_correlation' Dataframe
ethanol_correlation['Wage'] = df_merged['Valor_Anual']

In [ ]:
# Defining the new column names of the dataframe
ethanol_correlation.columns = ['Data', 'Ethanol', 'Ano', 'Wage']

In [ ]:
# removing the 'Ano' column (unescessary)
ethanol_correlation.drop('Ano', axis=1, inplace=True)

In [ ]:
ethanol_correlation 

In [ ]:
# Getting the number of Liters of ethanol one can afford with one minimum wage
ethanol_correlation['liters per wage'] = ethanol_correlation['Wage'] / ethanol_correlation['Ethanol']

In [ ]:
ethanol_correlation.head(30)

In [ ]:
# Create a new dataframe with the 'Data' and 'liters per wage' column
ethanol_per_wage = ethanol_correlation[['Data', 'liters per wage']]

In [ ]:
ethanol_per_wage

# Plotting Liters per minimum wage

In [ ]:
# Define the figure size
plt.figure(figsize=(12, 6))

# Plot the 'liters per wage' for gasoline over the years
plt.plot(gas_per_wage['Data'], gas_per_wage['liters per wage'], label='Gasolina')

# Plot the 'liters per wage' for ethanol over the years
plt.plot(ethanol_per_wage['Data'], ethanol_per_wage['liters per wage'], label='Etanol')


# Set the title of the plot
plt.title('Liters per minimum wage over the years')

# Set the label for the x-axis
plt.xlabel('Year')

# Set the label for the y-axis
plt.ylabel('Liters per minimum wage')

# Add a legend to the plot
plt.legend()

# Enable the grid for better visualization
plt.grid(True)


# Display the plot
plt.show()


# Plotting price over years

In [ ]:
# Define the figure size
plt.figure(figsize=(12, 6))

# Plot the price of gasoline over the years
plt.plot(gas_correlation['Data'], gas_correlation['Gas'], label='Gasolina')

# Plot the price of ethanol over the years
plt.plot(ethanol_correlation['Data'], ethanol_correlation['Ethanol'], label='Etanol')


# Set the title of the plot
plt.title('Price over the year')

# Set the label for the x-axis
plt.xlabel('Year')

# Set the label for the y-axis
plt.ylabel('Price')

# Add a legend to the plot
plt.legend()

# Enable the grid
plt.grid(True)


# Display the plot
plt.show()
